# **NOTEBOOK TEMPORAIRE DE CLASSEMENT ET CALCUL DES RESULTATS ATTENDUS PAR BM**

In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\marti\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\marti\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\marti\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Polygon, box
from shapely.ops import nearest_points
import re
from copy import copy, deepcopy

pd.set_option('display.max_columns', 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Idée générale : 
affecter chaque troncon à une commune pour l'agregation par commune, puis chaque troncon à un secteur pour l'agreg par secteur.
Ensuite on calcul le veh_km en se basant sur l'attribut "groupe" qui separent les troncons utilisés pour le calcul de la longueur ou non

Données Necessaires : 
- le fichier de trafic linearise
- le fichier des communes
- le fichier des secteurs

In [5]:
#import des données de base : voies cat1234 linéarisées et fichiers de communes
trafic_lin=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')
trafic_tot=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\affectationComplete.shp')
communes=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\communes_BdxM_2.shp',encoding='UTF-8')
communes['geom_c']=communes.geometry
secteurs=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\grands_secteurs.shp',encoding='UTF-8')
secteurs['geom_s']=secteurs.geometry
secteurs.rename(columns={'nom':'nom_sect'},inplace=True)

In [8]:
#on isole les cat_rhv 1,2,3
#trafic_cat1213=trafic_lin.loc[trafic_lin['cat_rhv'].isin(['1','2','3','33','61','62','63'])].copy()
# et on n egarde que les lignes d'attribut groupe == 1 (qui servent aux calculs de la longueur
trafic_tot_grp1=trafic_tot.loc[trafic_tot['groupe']==1].copy()
trafic_tot_grp1['longueur']=trafic_tot_grp1.geometry.length/1000

In [9]:
#jointure entre les lignes et les communes
joint_lin_com=gp.sjoin(trafic_tot_grp1[['ident','geometry','cat_rhv','groupe','numero', 'tmjo_2_sen','type_cpt','id_cpt_exp','longueur']], communes[['INSEE_COM','NOM','geometry', 'geom_c']],how='left',op='intersects')
joint_lin_com=joint_lin_com.loc[~pd.isnull(joint_lin_com.geom_c)].copy()

In [14]:
#calcul des longueurs et pourcentage de recouvrement
joint_lin_com['lg_intersect']=joint_lin_com.apply(lambda x : x['geometry'].intersection(x['geom_c']).length, axis=1)
joint_lin_com['lg_lgn']=joint_lin_com.geometry.length
joint_lin_com['ratio_lg']=joint_lin_com.lg_intersect/joint_lin_com.lg_lgn
#on ne conserve que le max des recouvrements pour chaque ligne
corresp_ident_commune=joint_lin_com.loc[joint_lin_com.ratio_lg==joint_lin_com.groupby('ident').ratio_lg.transform(max)].copy()

In [15]:
#jointure entre les lignes et les secteurs
joint_lin_sect=gp.sjoin(corresp_ident_commune.drop('index_right',axis=1), secteurs,how='left',op='intersects')
joint_lin_sect=joint_lin_sect.loc[~pd.isnull(joint_lin_sect.geom_s)].copy()

In [18]:
#calcul des longueurs et pourcentage de recouvrement
joint_lin_sect['lg_intersect']=joint_lin_sect.apply(lambda x : x['geometry'].intersection(x['geom_s']).length, axis=1)
joint_lin_sect['lg_lgn']=joint_lin_sect.geometry.length
joint_lin_sect['ratio_lg']=joint_lin_sect.lg_intersect/joint_lin_sect.lg_lgn
#on ne conserve que le max des recouvrements pour chaque ligne
corresp_ident_sect=joint_lin_sect.loc[joint_lin_sect.ratio_lg==joint_lin_sect.groupby('ident').ratio_lg.transform(max)].copy()
corresp_ident_sect.drop_duplicates('ident',inplace=True)

In [20]:
#on ne garde que les attributs qui nous interesse et on calcul le veh_km
pour_calcul=corresp_ident_sect[['geometry','ident','cat_rhv','groupe','numero', 'tmjo_2_sen','type_cpt','id_cpt_exp','longueur','NOM','nom_sect']].copy()
pour_calcul['veh_km']=pour_calcul.tmjo_2_sen*pour_calcul.longueur
pour_calcul=pour_calcul.loc[~pour_calcul.cat_rhv.isna()]
pour_calcul['cat_rhv_simplifiee']=pour_calcul.cat_rhv.apply(lambda x : str(int(x)%10))

In [ ]:
#modofier attribut_secteur
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Rocade_N-O' in x['numero'], axis=1),'nom_sect']='rocade NO'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Rocade_RD' in x['numero'], axis=1),'nom_sect']='rocade RD'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Rocade_Sud' in x['numero'], axis=1),'nom_sect']='rocade Sud'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('rocade|autoroute a 630|pont d\'aquitaine|pont francois mitterrand',x['numero'].lower()) else False) 
                    and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='rocade bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a 10',x['numero'].lower()) else False) 
                    and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='A10 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a 10',x['numero'].lower()) else False) 
                    and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='A10'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('route nationale 89',x['numero'].lower()) else False) 
                                and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='N89 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('route nationale 89',x['numero'].lower()) else False) 
                                and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='N89'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('autoroute a 63$',x['numero'].lower()) else False) 
                                and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='A63 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('autoroute a 63$',x['numero'].lower()) else False) 
                                and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='A63'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a62',x['numero'].lower()) else False) 
                                and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='A62 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a62',x['numero'].lower()) else False) 
                                and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='A62'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Boulevards_rive_gauche' in x['numero'], axis=1),'nom_sect']='Bld RG'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Boulevards_rive_droite' in x['numero'], axis=1),'nom_sect']='Bld RD'

In [26]:
pour_calcul.to_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\pour_calcul.shp')

In [27]:
#stat communales hors reseau Etat ou SCA et Blds
pour_calcul_hors_GI=pour_calcul.loc[pour_calcul.apply(lambda x : True if not re.match('A.|N89|roc|Bld',x['nom_sect']) else False, axis=1)].copy()
stat_commune=pour_calcul_hors_GI.groupby(['NOM','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})

#grandes infras
#stats Blds
blds=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : x[:3]=='Bld')].copy()
stats_blds=blds.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
#stats Rocade et SCA
Rocade_rd_A10_N89=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : x in ('A10','N89','rocade RD'))].copy()
stat_Rocade_rd_A10_N89=Rocade_rd_A10_N89.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
Rocade_sud_A63_A62=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : x in ('A62','A63','rocade Sud'))].copy()
stat_Rocade_sud_A63_A62=Rocade_sud_A63_A62.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
Rocade_NO=pour_calcul.loc[pour_calcul.nom_sect=='rocade NO'].copy()
stat_Rocade_NO=Rocade_NO.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
#pour info
bretelles=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : 'bretelle' in x )].copy()
stat_bretelles=bretelles.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
#concat GI
grandes_infras=pd.concat([stats_blds,stat_Rocade_rd_A10_N89,stat_Rocade_sud_A63_A62,stat_Rocade_NO,stat_bretelles],axis=0, sort=False)

#stats secteurs
stat_secteurs=pour_calcul_hors_GI.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})

#donnees_generales
donnees_generales=pour_calcul.groupby(['cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})

In [448]:
#export resultats excel
with pd.ExcelWriter(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\cat123\BM_stat_veh-km_cat123_2.xls') as writer:
    donnees_generales.to_excel(writer, sheet_name='Donnees_generales')
    grandes_infras.reset_index().to_excel(writer, sheet_name='Grandes_infras')
    stat_secteurs.reset_index().to_excel(writer, sheet_name='Secteurs')
    stat_commune.reset_index().to_excel(writer, sheet_name='Communes')

In [28]:
stat_secteurs.sum()

veh_km      9.966821e+06
longueur    3.517658e+03
dtype: float64

In [29]:
stat_commune.sum()

veh_km      9.966821e+06
longueur    3.517658e+03
dtype: float64

In [31]:
filaire_source=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\Filaire_voie\FV_TRONC_L.shp')
#export fichier complet
trafic_tot[['ident','geometry','cat_rhv','groupe','tmjo_2_sen','type_cpt','id_cpt_exp']].merge(
    filaire_source[['IDENT','NOM_VOIE']], left_on='ident', right_on='IDENT').rename(columns=
    {'id_cpt_exp':'id_comptag'}).drop('IDENT', axis=1).to_file(
r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\FV_affectation_complete.shp')

In [32]:
#export fichier de calcul longueur
pour_calcul[['ident','geometry','cat_rhv','groupe','tmjo_2_sen','type_cpt','id_cpt_exp','longueur','NOM','nom_sect','veh_km']].merge(
    filaire_source[['IDENT','NOM_VOIE']], left_on='ident', right_on='IDENT').rename(columns=
    {'id_cpt_exp':'id_comptag', 'NOM':'nom_comm'}).drop('IDENT', axis=1).to_file(
r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\FV_affectation_complete_grp1.shp')